# Small mIoU metric

In [ ]:
import numpy as np
from skimage import io, img_as_bool, measure, morphology
import matplotlib.pyplot as plt

## Working example

In [ ]:
gt_file = '/Users/avaimar/Desktop/m_3712141_se_10_060_20200525_134.npy'
mask_file = '/Users/avaimar/Desktop/m_3712141_se_10_060_20200525_134_mask.npy'
mask_wt_file = '/Users/avaimar/Desktop/m_3712141_se_10_060_20200525_134_mask_wt.npy'
pred_file = '/Users/avaimar/Desktop/p2_sj_adu_0.2_1.03_miou.npy'

In [ ]:
image = np.load(gt_file)
pred_image = np.load(pred_file)
mask = np.load(mask_file)

gt_image = (mask > 0).astype(np.int32)
gt_area = mask

In [ ]:
print('Ground truth')
plt.imshow(image)

In [ ]:
print('Mask')
plt.imshow(gt_image)

In [ ]:
print('Inference')
plt.imshow(pred_image)

In [ ]:
print('Area')
plt.imshow(gt_area)

In [ ]:
# Get small buildings (from ground truth)
small_build_gt = ((gt_area < 115.) & (gt_area > 15)).astype(bool)

print('Small gt buildings')
plt.imshow(small_build_gt)

## Helper functions

In [ ]:
# Pad small gt buildings
def pad_small_buildings(small_build_gt):
    # shapely buffer?
    pass # TODO

In [ ]:
# Get predicted small buildings
def get_pred_small_buildings(pred_image):
    
    # TODO CONVERT TO SHAPELY USING COORDINATES
    
    # TODO: SEPARATE CLOSELY CONNECTED BUILDINGS
    
    # TODO: FILTER OUT PREDICTIONS ON ROADS
    
    labels = measure.label(pred_image)

    clusters = []
    for i in range(1, len(np.unique(labels))):
        clusters.append(np.column_stack(np.where(labels == i)))

    pred_small_build = np.zeros_like(pred_image, dtype=float)
    for cluster in clusters:
        # Get single building array
        row_indices, col_indices = cluster
        ar = np.zeros((pred_image.shape))
        ar[row_indices, col_indices] = 1
        
        # Measure building area
        a = measure.regionprops(ar.astype(np.int32))
        # calibrate with gt area measure??
        
        # 
        area = None
        if THRESH < area < THRESH: # TODO FILTER PREDICTIONS THAT ARE TOO SMALL
            for c in cluster:
                pred_small_build[c[0], c[1]] = 1.

    return pred_small_build # TODO

## Main function

In [ ]:
def small_mIOU(self, gt_image, pred_image, gt_area):
    len_batch = len(gt_image)
    classes = np.unique(gt_image)

    mIOU = []

    def compute_iou(gt, pred):
        overlap = np.sum(np.logical_and(gt, pred))
        n_gt = np.sum(gt)
        n_pred = np.sum(pred)
        return (overlap) / (n_gt + n_pred - overlap)

    # Get gt and predictions for each class
    bg_gt, bg_pred = (gt_image == 0).astype(np.int32), (pred_image == 0).astype(np.int32)
    build_gt, build_pred = (gt_image == 1).astype(np.int32), (pred_image == 1).astype(np.int32)
    
    # Get TIF from file name, get bounding box
    # TODO 

    # Class 0 (bg) ---------------------
    # * Get gt small buildings
    small_build_gt = ((gt_area < 115.) & (gt_area > 15)).astype(bool)

    # * Pad gt small buildings
    padded_small_build_gt = pad_small_buildings(small_build_gt)

    # * Get predicted small buildings
    small_build_pred = get_pred_small_buildings(build_pred)

    # Invert
    bg_mask = ((padded_small_build_gt + small_build_pred) > 0)
    bg_mask = 1 - bg_mask

    # * Mask predictions with padded gt small buildings and predicted small buildings
    masked_bg_pred = np.multiply(bg_pred, bg_mask)
    masked_bg_gt = np.multiply(bg_gt, bg_mask)

    bg_iou = compute_iou(masked_bg_gt, masked_bg_pred)
    mIOU.append(bg_iou)

    # Class 1 (building) ---------------------
    # * Mask predictions with small buildings
    small_build_pred = np.multiply(build_pred, small_build_gt)

    # * Compute mIOU with adjusted gt/pred
    build_iou = compute_iou(small_build_gt, small_build_pred)
    mIOU.append(build_iou)

    return sum(mIOU) / len(classes)